In [81]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import classification_report

from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Dense, Flatten
from keras.models import Sequential

### 자연어 처리(Natural Language Processing, NLP)
자연어란 우리가 평소에 말하는 음성이나 텍스트를 말한다. 즉 자연어 처리는 이러한 음성이나 텍스트를 컴퓨터가 인식하고 처리하는 것을 말한다

#### 텍스트 토큰화
- 입력할 텍스트가 준비되면 이를 단어별, 문장별, 형태소별로 나눌 수 있는데, 이렇게 작게 나누어진 하나의 단위를 토큰(token)이라고 부른다.
- 입력된 텍스트를 잘게 나누는 과정을 토큰화(tokenization)라고 한다.

In [2]:
# 문장을 단어로 나누기
text = '해보지 않으면 해낼 수 없다'
result = text_to_word_sequence(text)
result

['해보지', '않으면', '해낼', '수', '없다']

In [4]:
# 단어 빈도 계산
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']
token = Tokenizer()
token.fit_on_texts(docs)
token.word_counts

OrderedDict([('먼저', 1),
             ('텍스트의', 2),
             ('각', 1),
             ('단어를', 1),
             ('나누어', 1),
             ('토큰화합니다', 1),
             ('단어로', 1),
             ('토큰화해야', 1),
             ('딥러닝에서', 2),
             ('인식됩니다', 1),
             ('토큰화한', 1),
             ('결과는', 1),
             ('사용할', 1),
             ('수', 1),
             ('있습니다', 1)])

In [6]:
# 총 몇 개의 문장이 들어있는지 보기
token.document_count

3

In [8]:
# 각 단어들이 몇 개의 문장에 나오는지 보기
token.word_docs

defaultdict(int,
            {'먼저': 1,
             '텍스트의': 2,
             '나누어': 1,
             '단어를': 1,
             '토큰화합니다': 1,
             '각': 1,
             '딥러닝에서': 2,
             '토큰화해야': 1,
             '인식됩니다': 1,
             '단어로': 1,
             '있습니다': 1,
             '결과는': 1,
             '토큰화한': 1,
             '수': 1,
             '사용할': 1})

In [9]:
# 각 단어에 매겨진 인덱스 값 보기
token.word_index

{'텍스트의': 1,
 '딥러닝에서': 2,
 '먼저': 3,
 '각': 4,
 '단어를': 5,
 '나누어': 6,
 '토큰화합니다': 7,
 '단어로': 8,
 '토큰화해야': 9,
 '인식됩니다': 10,
 '토큰화한': 11,
 '결과는': 12,
 '사용할': 13,
 '수': 14,
 '있습니다': 15}

#### 단어 원-핫 인코딩
- 각 단어를 모두 0으로 바꿔주고 원하는 단어만 1로 바꾼다.
- 파이썬 배열의 인덱스는 0부터 시작하므로, 맨 앞에 0이 추가되는 것에 주의한다.

In [14]:
token = Tokenizer()
# 입력을 배열로
token.fit_on_texts([text])
# 각 단어의 인덱스 값
token.word_index

{'해보지': 1, '않으면': 2, '해낼': 3, '수': 4, '없다': 5}

In [15]:
# 인덱스로만 채워진 새로운 배열 생성
x = token.texts_to_sequences([text])
x

[[1, 2, 3, 4, 5]]

In [18]:
# 원-핫 인코딩
# 첫번째 인덱스 값은 0
x = to_categorical(x)
x

array([[[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]]], dtype=float32)

#### 단어 임베딩
- 원-핫 인코딩을 사용하면 벡터의 길이가 너무 길어진다는 단점이 있다.
- 이러한 공간적 낭비를 해결하기 위해 등장한 것이 단어 임베딩(word embedding)이라는 방법이다
- 주어진 배열을 정해진 길이로 압축시킨다.
- 각 단어 간의 유사도를 계산하여 각 배열을 새로운 수치로 바꿔준다
- ex) Embedding(16, 4)
    - 16: 입력될 총 단어 수
    - 4: 임베딩 후 출력되는 벡터 크기

### 텍스트 읽고 긍정, 부정 예측하기

In [48]:
# 텍스트 리뷰 자료
docs = ['너무 재밌네요', '최고예요', '참 잘 만든 영화예요', '추천하고 싶은 영화입니다.', '한 번 더 보고싶네요',
        '글쎄요', '별로예요', '생각보다 지루하네요', '연기가 어색해요', '재미없어요']

# 긍정 리뷰 1, 부정 리뷰 0으로 클래스 지정
labels = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [49]:
# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
token.word_index

{'너무': 1,
 '재밌네요': 2,
 '최고예요': 3,
 '참': 4,
 '잘': 5,
 '만든': 6,
 '영화예요': 7,
 '추천하고': 8,
 '싶은': 9,
 '영화입니다': 10,
 '한': 11,
 '번': 12,
 '더': 13,
 '보고싶네요': 14,
 '글쎄요': 15,
 '별로예요': 16,
 '생각보다': 17,
 '지루하네요': 18,
 '연기가': 19,
 '어색해요': 20,
 '재미없어요': 21}

In [50]:
# 토큰에 지정된 인덱스로 새로운 배열 생성
x = token.texts_to_sequences(docs)
x

[[1, 2],
 [3],
 [4, 5, 6, 7],
 [8, 9, 10],
 [11, 12, 13, 14],
 [15],
 [16],
 [17, 18],
 [19, 20],
 [21]]

In [51]:
# 패딩 작업(padding, 길이를 똑같이 맞춰 주는 작업)
padded_x = pad_sequences(x)
padded_x

array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 4,  5,  6,  7],
       [ 0,  8,  9, 10],
       [11, 12, 13, 14],
       [ 0,  0,  0, 15],
       [ 0,  0,  0, 16],
       [ 0,  0, 17, 18],
       [ 0,  0, 19, 20],
       [ 0,  0,  0, 21]], dtype=int32)

In [52]:
# 단어 수 
# 전체 단어의 맨 앞에 0이 먼저 나와야 하므로 총 단어 수 더하기 1
word_size = len(token.word_index) + 1

In [53]:
# seed 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [54]:
# 모델 설정
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [55]:
# 모델 컴파일
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [56]:
# 모델 실행
model.fit(padded_x, labels, epochs=20)

Epoch 1/20
1/1 [==============================] - 1s 841ms/step - loss: 0.6966 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 17ms/step - loss: 0.6940 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6915 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6891 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6866 - accuracy: 0.6000
Epoch 6/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6841 - accuracy: 0.6000
Epoch 7/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6817 - accuracy: 0.6000
Epoch 8/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6792 - accuracy: 0.6000
Epoch 9/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6768 - accuracy: 0.7000
Epoch 10/20
1/1 [==============================] - 0s 20ms/step - loss: 0.6743 - accuracy: 0.8000
Epoch 11/20
1/1 [==============

In [60]:
# 정확도
model.evaluate(padded_x, labels)[1]

1/1 [==============================] - 0s 169ms/step - loss: 0.6477 - accuracy: 1.0000


1.0

In [78]:
# 예측
y_pred = (model.predict(padded_x) > 0.5).astype(int)
y_pred = y_pred.reshape(y_pred.shape[0])
y_pred

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [80]:
# confusion matrix
pd.crosstab(labels, y_pred, rownames=['Actual'], colnames=['Predicted'])

Predicted,0,1
Actual,,
0,5,0
1,0,5


In [84]:
# score
print(classification_report(labels, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



긍정과 부정 리뷰 모두 잘 예측함을 확인 할 수 있다